# Lab 1: Introduction and Kubernetes-Based Deployment

## Overview

In this lab, you will:
- Set up your personal namespace in the shared Kubernetes cluster
- Deploy Dynamo using namespace-scoped operator on Kubernetes
- Configure a backend engine using aggregated serving
- Test the deployment with OpenAI-compatible API
- Benchmark the deployment using AI-Perf

## Duration: ~90 minutes

---

## Section 1: Environment Setup

### Objectives
- Verify Kubernetes access (shared cluster)
- Create your personal namespace
- Install Dynamo dependencies in your namespace
- Set up prerequisites (kubectl, helm)

### Prerequisites
Before starting, ensure you have:
- ✅ Kubernetes cluster access (kubeconfig provided by instructor)
- ✅ `kubectl` installed (version 1.24+) or `microk8s kubectl`
- ✅ `helm` 3.x installed
- ✅ HuggingFace token from [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

### Step 2: Set Configuration Variables

Set your configuration variables. **Replace the values below with your own:**


In [ ]:
import os

# Set lab configuration
os.environ['RELEASE_VERSION'] = '0.7.1'
os.environ['NAMESPACE'] = 'dynamo-lab1'
os.environ['HF_TOKEN'] = ''  # Will be set securely in a later cell
os.environ['CACHE_PATH'] = '/data/huggingface-cache'  # Local cache path

NAMESPACE = os.environ['NAMESPACE']
FRONTEND_PORT = '10000'
GRAFANA_PORT = '30080'  # NodePort from monitoring stack

print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print("🎓 Lab 1: Environment Configuration")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"  Release Version:  {os.environ['RELEASE_VERSION']}")
print(f"  Namespace:        {NAMESPACE}")
print(f"  Cache Path:       {os.environ['CACHE_PATH']}")
print("")
print("📌 Access Ports:")
print(f"  Frontend API:     localhost:{FRONTEND_PORT} (via port-forward)")
print(f"  Grafana:          http://<node-ip>:{GRAFANA_PORT}")
print("")
print("💡 Use port-forward to access the frontend:")
print(f"   kubectl port-forward -n {NAMESPACE} deployment/vllm-agg-router-frontend {FRONTEND_PORT}:8000")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🎓 Lab 1: Environment Configuration
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
  Release Version:  0.7.1
  Namespace:        dynamo-lab1
  Cache Path:       /data/huggingface-cache

📌 Access Ports:
  Frontend API:     localhost:10000 (via port-forward)
  Grafana:          http://<node-ip>:30080

💡 Use port-forward to access the frontend:
   kubectl port-forward -n dynamo-lab1 deployment/vllm-agg-router-frontend 10000:8000
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


### Step 3: Verify Kubernetes Access


In [2]:
%%bash
# Verify kubectl is installed and configured
kubectl version --client

# Check cluster connection
kubectl cluster-info

# Check GPU nodes are available (optional)
kubectl get nodes -o custom-columns=NAME:.metadata.name,GPUs:.status.capacity.nvidia\\.com/gpu

Client Version: v1.35.0
Kustomize Version: v5.7.1
Kubernetes control plane is running at https://172.27.37.39:16443
CoreDNS is running at https://172.27.37.39:16443/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
NAME             GPUs
brev-cg71i9ose   2


### Step 4: Set Up NGC Authentication

To access NVIDIA's Dynamo container images, you need to authenticate with NGC (NVIDIA GPU Cloud).

#### Get Your NGC API Key

1. Go to [NGC](https://ngc.nvidia.com/)
2. Sign in or create an account
3. Click on your profile in the top right corner
4. Select **"Setup"** → **"Generate API Key"**
5. Copy your API key (it will only be shown once!)

#### Login to NGC Container Registry


In [4]:
import os
import getpass

# Get NGC API key from user
print("Enter your NGC API Key from https://ngc.nvidia.com/")
print("(Go to Profile > Setup > Generate API Key)")
print("")
NGC_API_KEY = getpass.getpass("NGC API Key: ")

# Save it for later use (creating pull secrets)
os.environ['NGC_API_KEY'] = NGC_API_KEY

print("")
print("✓ NGC API key saved")
print("  You can now use it to login and create pull secrets")

Enter your NGC API Key from https://ngc.nvidia.com/
(Go to Profile > Setup > Generate API Key)



NGC API Key:  ········



✓ NGC API key saved
  You can now use it to login and create pull secrets


#### Set HuggingFace Token

HuggingFace token is required to download models. Get yours from https://huggingface.co/settings/tokens

In [ ]:
import os
import getpass

# Get HuggingFace token from user
print("Enter your HuggingFace Token from https://huggingface.co/settings/tokens")
print("(Create a 'Read' token if you don't have one)")
print("")
HF_TOKEN = getpass.getpass("HF Token: ")

# Save to environment
os.environ['HF_TOKEN'] = HF_TOKEN
print("✓ HuggingFace token saved to environment")

#### Login to NGC Registry

In [17]:
import os
import subprocess

# Login to NGC container registry
ngc_key = os.environ.get('NGC_API_KEY')
result = subprocess.run(
    ['helm', 'registry', 'login', 'nvcr.io', '--username', '$oauthtoken', '--password-stdin'],
    input=ngc_key,
    text=True,
    capture_output=True
)
print(result.stdout)
if result.returncode != 0:
    print(result.stderr)

print("")
print("✓ NGC authentication complete")
print("  You can now pull Dynamo container images")



✓ NGC authentication complete
  You can now pull Dynamo container images


### Step 5: Create Your Namespace

In [6]:
import os
import subprocess

# Create the namespace
namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')
result = subprocess.run(['kubectl', 'create', 'namespace', namespace], 
                       capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0 and "AlreadyExists" not in result.stderr:
    print(result.stderr)

# Verify namespace was created
result = subprocess.run(['kubectl', 'get', 'namespace', namespace], 
                       capture_output=True, text=True)
print(result.stdout)


NAME          STATUS   AGE
dynamo-lab1   Active   62m



### Step 6: Create NGC Pull Secret

Create a Kubernetes secret so that pods can pull images from NGC.

In [7]:
import os
import subprocess

# Get variables
namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')
ngc_key = os.environ.get('NGC_API_KEY')

# Create NGC image pull secret
result = subprocess.run([
    'kubectl', 'create', 'secret', 'docker-registry', 'ngc-secret',
    '--docker-server=nvcr.io',
    '--docker-username=$oauthtoken',
    f'--docker-password={ngc_key}',
    '--namespace', namespace
], capture_output=True, text=True)

print(result.stdout)
if result.returncode != 0 and "AlreadyExists" not in result.stderr:
    print(result.stderr)
else:
    # Verify secret was created
    result = subprocess.run(['kubectl', 'get', 'secret', 'ngc-secret', '-n', namespace], 
                           capture_output=True, text=True)
    print(result.stdout)
    print(f"✓ NGC pull secret created in namespace: {namespace}")


error: failed to create secret secrets "ngc-secret" already exists



### Step 7: Create HuggingFace Token Secret

In [8]:
import os
import subprocess

# Get variables
namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')
hf_token = os.environ.get('HF_TOKEN', '')

# Create HuggingFace token secret
result = subprocess.run([
    'kubectl', 'create', 'secret', 'generic', 'hf-token-secret',
    f'--from-literal=HF_TOKEN={hf_token}',
    '--namespace', namespace
], capture_output=True, text=True)

print(result.stdout)
if result.returncode != 0 and "AlreadyExists" not in result.stderr:
    print(result.stderr)
else:
    # Verify secret was created
    result = subprocess.run(['kubectl', 'get', 'secret', 'hf-token-secret', '-n', namespace], 
                           capture_output=True, text=True)
    print(result.stdout)
    print("✓ HuggingFace token secret created")


error: failed to create secret secrets "hf-token-secret" already exists



## Section 2: Install Dynamo Platform

### Objectives
- Install Dynamo CRDs (Custom Resource Definitions)
- Install Dynamo platform (etcd, NATS, operator) 
- Verify platform components are running

### Architecture
```
Client → Frontend → Router → Worker(s) with Backend Engine
                        ↓
                 etcd + NATS
                        ↓
                Dynamo Operator
```

### Deployment Mode

We're using the **recommended cluster-wide deployment** (default). According to the [official Dynamo documentation](https://github.com/ai-dynamo/dynamo/blob/main/deploy/helm/charts/platform/README.md):

- ✅ **Recommended**: One cluster-wide operator per cluster (default)
- This is the standard deployment for single-node and production clusters
- Install a **namespace-scoped Dynamo operator** that only manages resources in your namespace
- The CRDs are cluster-wide and should already be installed (check first)

### Step 1: Check if Dynamo CRDs Are Installed

**Note:** CRDs are cluster-wide resources and only need to be installed **once per cluster**. If already installed, skip to Step 2.


In [9]:
%%bash
# Check if CRDs already exist
if kubectl get crd dynamographdeployments.nvidia.com &>/dev/null && \
   kubectl get crd dynamocomponentdeployments.nvidia.com &>/dev/null; then
    echo "✓ CRDs already installed"
    kubectl get crd | grep nvidia.com
else
    echo "⚠️  CRDs not found. Ask instructor to install them, or proceed with Step 1b"
fi

⚠️  CRDs not found. Ask instructor to install them, or proceed with Step 1b


### Step 1b: Install CRDs (Optional - Instructor May Do This)

**Skip this step if CRDs are already installed.** If needed, run:


In [10]:
%%bash
# Install Dynamo CRDs (only if not already installed)
echo "Installing Dynamo CRDs..."
helm fetch https://helm.ngc.nvidia.com/nvidia/ai-dynamo/charts/dynamo-crds-$RELEASE_VERSION.tgz
helm install dynamo-crds dynamo-crds-$RELEASE_VERSION.tgz --namespace default

echo ""
echo "Verifying CRD installation:"
kubectl get crd | grep nvidia.com

Installing Dynamo CRDs...
NAME: dynamo-crds
LAST DEPLOYED: Thu Jan 15 22:16:57 2026
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None

Verifying CRD installation:
clusterpolicies.nvidia.com                            2026-01-15T20:11:12Z
dynamocomponentdeployments.nvidia.com                 2026-01-15T22:16:58Z
dynamographdeploymentrequests.nvidia.com              2026-01-15T22:16:58Z
dynamographdeployments.nvidia.com                     2026-01-15T22:16:58Z
dynamomodels.nvidia.com                               2026-01-15T22:16:58Z
nvidiadrivers.nvidia.com                              2026-01-15T20:11:12Z


### Step 2: Install Dynamo Platform

This installs ETCD, NATS, and the Dynamo Operator Controller (cluster-wide by default).


In [ ]:
import subprocess
import os

release_version = os.environ.get('RELEASE_VERSION', '0.7.1')
namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')

# Download platform chart
chart_url = f"https://helm.ngc.nvidia.com/nvidia/ai-dynamo/charts/dynamo-platform-{release_version}.tgz"
subprocess.run(['helm', 'fetch', chart_url])

# Install Dynamo platform (cluster-wide by default - recommended)
print(f"Installing Dynamo platform in namespace: {namespace}")
subprocess.run([
    'helm', 'install', 'dynamo-platform',
    f'dynamo-platform-{release_version}.tgz',
    '--namespace', namespace
])

print("")
print("Platform installation initiated. Waiting for pods to be ready...")

Installing Dynamo platform in namespace: dynamo-lab1
NAME: dynamo-platform
LAST DEPLOYED: Thu Jan 15 22:17:04 2026
NAMESPACE: dynamo-lab1
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
# SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Platform installation initiated. Waiting for pods to be ready...


### Step 3: Wait for Platform Pods to Be Ready

Re-run the following cell until all pods report as "Running"


In [21]:
import subprocess
import os

namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')

# Check pods in namespace
subprocess.run(['kubectl', 'get', 'pods', '-n', namespace])

NAME                                                              READY   STATUS    RESTARTS   AGE
dynamo-platform-dynamo-operator-controller-manager-79fd6d7ncf7r   2/2     Running   0          12m
dynamo-platform-etcd-0                                            1/1     Running   0          12m
dynamo-platform-nats-0                                            2/2     Running   0          12m


CompletedProcess(args=['kubectl', 'get', 'pods', '-n', 'dynamo-lab1'], returncode=0)

## Section 3: Deploy Your First Model with Aggregated Serving

### Objectives
- Understand aggregated serving architecture
- Configure and deploy a model using vLLM backend
- Use Kubernetes manifests to deploy Dynamo resources

### Available Backends
In this lab, we'll use **vLLM** with aggregated serving for simplicity:
- **vLLM**: High-throughput serving with PagedAttention
- Model: `Qwen/Qwen2.5-1.5B-Instruct` (small, fast to download)
- Architecture: Aggregated serving with KV-cache routing

**Other backends** (for exploration):
- **SGLang**: Optimized for complex prompting and structured generation
- **TensorRT-LLM**: Maximum performance on NVIDIA GPUs

### Deployment Configuration

We'll use a `DynamoGraphDeployment` resource that defines:
- **Frontend**: OpenAI-compatible API endpoint with KV-cache routing
- **Worker**: vLLM worker with 1 GPU running the model

### Step 1: Update the Deployment Configuration

Before deploying, we need to update the YAML configuration with your specific values:


In [22]:
%%bash
# Update agg_router.yaml with your configuration

# Replace my-tag with actual version
sed -i "s/my-tag/$RELEASE_VERSION/g" agg_router.yaml

# Replace cache path
sed -i "s|/YOUR/LOCAL/CACHE/FOLDER|$CACHE_PATH|g" agg_router.yaml

echo "✓ Configuration updated in agg_router.yaml"
echo ""
echo "Verify image tags (should show version, not my-tag):"
grep "image:" agg_router.yaml

✓ Configuration updated in agg_router.yaml

Verify image tags (should show version, not my-tag):
          image: nvcr.io/nvidia/ai-dynamo/vllm-runtime:0.7.0.post2
          image: nvcr.io/nvidia/ai-dynamo/vllm-runtime:0.7.0.post2


### Step 2: Deploy the Model


In [24]:
import subprocess
import os

namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')

# Apply the deployment
result = subprocess.run(
    ['kubectl', 'apply', '-f', 'agg_router.yaml', '--namespace', namespace],
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print(result.stderr)

print("")
print("✓ Deployment created. This will take 4-6 minutes for first run.")
print("  - Pulling container images")
print("  - Downloading model from HuggingFace")
print("  - Loading model into GPU memory")

dynamographdeployment.nvidia.com/vllm-agg-router unchanged
service/vllm-frontend-nodeport unchanged


✓ Deployment created. This will take 4-6 minutes for first run.
  - Pulling container images
  - Downloading model from HuggingFace
  - Loading model into GPU memory


### Step 3: Monitor Deployment Progress


In [25]:
import subprocess
import os

namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')

# Check deployment status
print("Checking DynamoGraphDeployment status:")
subprocess.run(['kubectl', 'get', 'dynamographdeployment', '-n', namespace])

print("\nPod status (wait for all pods to be 1/1 Ready):")
result = subprocess.run(
    ['kubectl', 'get', 'pods', '-n', namespace],
    capture_output=True,
    text=True
)
# Filter for vllm pods
for line in result.stdout.split('\n'):
    if 'vllm' in line.lower() or 'NAME' in line:
        print(line)

print("\n# To watch in real-time, run: kubectl get pods -n", namespace, "-w")

Checking DynamoGraphDeployment status:
NAME              AGE
vllm-agg-router   7m15s

Pod status (wait for all pods to be 1/1 Ready):
NAME                                                              READY   STATUS    RESTARTS   AGE
vllm-agg-router-frontend-947cb44fd-xv26s                          1/1     Running   0          7m15s
vllm-agg-router-vllmdecodeworker-5685d757-v8tgk                   1/1     Running   0          7m15s

# To watch in real-time, run: kubectl get pods -n dynamo-lab1 -w


### Step 4: View Worker Logs (Optional)

While waiting for the deployment, you can watch the model loading progress:


In [26]:
import subprocess
import os

namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')

# Get logs from the worker pod
result = subprocess.run(
    ['kubectl', 'get', 'pods', '-n', namespace],
    capture_output=True,
    text=True
)

worker_pod = None
for line in result.stdout.split('\n'):
    if 'vllmdecodeworker' in line.lower():
        worker_pod = line.split()[0]
        break

if worker_pod:
    print(f"Viewing logs from: {worker_pod}")
    print("Look for:")
    print("  - 'Loading model weights...' (downloading)")
    print("  - 'Model loading took X.XX GiB' (loaded)")
    print("")
    subprocess.run(['kubectl', 'logs', worker_pod, '-n', namespace, '--tail=30'])
else:
    print("Worker pod not found yet, please wait and try again")

Viewing logs from: vllm-agg-router-vllmdecodeworker-5685d757-v8tgk
Look for:
  - 'Loading model weights...' (downloading)
  - 'Model loading took X.XX GiB' (loaded)

2026-01-15T22:33:02.564145Z  INFO core._initialize_kv_caches: init engine (profile, create kv cache, warmup model) took 16.15 seconds
2026-01-15T22:33:02.842714Z  INFO nixl_connector: NIXL is available
2026-01-15T22:33:02.844159Z  INFO factory.create_connector: Creating v1 connector with name: NixlConnector and engine_id: 3ac48b16-3f97-41a9-b6a2-f278ad968c62
2026-01-15T22:33:02.844192Z  WARN base.__init__: Initializing KVConnectorBase_V1. This API is experimental and subject to change in the future as we iterate the design.
2026-01-15T22:33:02.844229Z  INFO nixl_connector.__init__: Initializing NIXL Scheduler 3ac48b16-3f97-41a9-b6a2-f278ad968c62
2026-01-15T22:33:02.844663Z  INFO kv_events.__init__: Starting ZMQ publisher thread
2026-01-15T22:33:03.382392Z  INFO dynamo_llm::kv_router::publisher: Registered KvStats Prometheu

## Section 4: Testing and Validation

### Objectives
- Expose the service locally using port forwarding
- Send test requests to the deployment
- Verify OpenAI API compatibility
- Test streaming and non-streaming responses

### Testing Strategy
Once your deployment is running (`1/1 Ready`), you'll:
1. Forward the frontend service port to localhost
2. Test with curl commands
3. Verify response format and functionality

### Step 1: Set Up Port Forwarding

Forward the service port to localhost (run in background):


In [27]:
import subprocess

# Get the node IP
result = subprocess.run(
    ['kubectl', 'get', 'nodes', '-o', 'jsonpath={.items[0].status.addresses[?(@.type=="InternalIP")].address}'],
    capture_output=True,
    text=True
)

node_ip = result.stdout.strip()
print(f"Node IP: {node_ip}")
print("")
print(f"Frontend URL: http://{node_ip}:30100")
print("")
print(f"✓ Access the frontend at: http://{node_ip}:30100")

Node IP: 172.27.37.39

Frontend URL: http://172.27.37.39:30100

✓ Access the frontend at: http://172.27.37.39:30100


### Step 2: Test the `/v1/models` Endpoint


In [2]:
import subprocess

# Get node IP
result = subprocess.run(
    ['kubectl', 'get', 'nodes', '-o', 'jsonpath={.items[0].status.addresses[?(@.type=="InternalIP")].address}'],
    capture_output=True,
    text=True
)
node_ip = result.stdout.strip()

# Test the /v1/models endpoint
subprocess.run(['curl', f'http://{node_ip}:30100/v1/models'])

{"object":"list","data":[{"id":"Qwen/Qwen2.5-1.5B-Instruct","object":"object","created":1768517087,"owned_by":"nvidia"}]}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   121  100   121    0     0   165k      0 --:--:-- --:--:-- --:--:--  118k


CompletedProcess(args=['curl', 'http://172.27.37.39:30100/v1/models'], returncode=0)

### Step 3: Simple Non-Streaming Chat Completion


In [29]:
import subprocess

# Get node IP
result = subprocess.run(
    ['kubectl', 'get', 'nodes', '-o', 'jsonpath={.items[0].status.addresses[?(@.type=="InternalIP")].address}'],
    capture_output=True,
    text=True
)
node_ip = result.stdout.strip()

# Test non-streaming chat completion
subprocess.run([
    'curl', f'http://{node_ip}:30100/v1/chat/completions',
    '-H', 'Content-Type: application/json',
    '-d', '''{
    "model": "Qwen/Qwen2.5-1.5B-Instruct",
    "messages": [{"role": "user", "content": "Hello! How are you?"}],
    "stream": false,
    "max_tokens": 50
  }'''
])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

{"id":"chatcmpl-dec8d3fa-cc4f-45b7-93f2-618d4b7633de","choices":[{"index":0,"message":{"content":"Hello! I'm just an artificial intelligence and don't have feelings or emotions like humans do. However, I'm here to help you with any questions or tasks you might have. Is there anything specific you'd like assistance with today?","role":"assistant","reasoning_content":null},"finish_reason":"stop"}],"created":1768516776,"model":"Qwen/Qwen2.5-1.5B-Instruct","object":"chat.completion","usage":{"prompt_tokens":35,"completion_tokens":48,"total_tokens":83}}

100   714  100   554  100   160   1785    515 --:--:-- --:--:-- --:--:--  2303


CompletedProcess(args=['curl', 'http://172.27.37.39:30100/v1/chat/completions', '-H', 'Content-Type: application/json', '-d', '{\n    "model": "Qwen/Qwen2.5-1.5B-Instruct",\n    "messages": [{"role": "user", "content": "Hello! How are you?"}],\n    "stream": false,\n    "max_tokens": 50\n  }'], returncode=0)

### Step 4: Test Streaming Response


In [30]:
import subprocess

# Get node IP
result = subprocess.run(
    ['kubectl', 'get', 'nodes', '-o', 'jsonpath={.items[0].status.addresses[?(@.type=="InternalIP")].address}'],
    capture_output=True,
    text=True
)
node_ip = result.stdout.strip()

# Test streaming chat completion
subprocess.run([
    'curl', f'http://{node_ip}:30100/v1/chat/completions',
    '-H', 'Content-Type: application/json',
    '-d', '''{
    "model": "Qwen/Qwen2.5-1.5B-Instruct",
    "messages": [{"role": "user", "content": "Write a short poem about AI"}],
    "stream": true,
    "max_tokens": 100
  }'''
])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

data: {"id":"chatcmpl-a18b42fb-2084-4da5-99c0-467fdb5a1ae1","choices":[{"index":0,"delta":{"content":"Art","function_call":null,"tool_calls":null,"role":"assistant","refusal":null,"reasoning_content":null}}],"created":1768516782,"model":"Qwen/Qwen2.5-1.5B-Instruct","service_tier":null,"system_fingerprint":null,"object":"chat.completion.chunk","usage":null}

data: {"id":"chatcmpl-a18b42fb-2084-4da5-99c0-467fdb5a1ae1","choices":[{"index":0,"delta":{"content":"ificial","function_call":null,"tool_calls":null,"role":"assistant","refusal":null,"reasoning_content":null}}],"created":1768516782,"model":"Qwen/Qwen2.5-1.5B-Instruct","service_tier":null,"system_fingerprint":null,"object":"chat.completion.chunk","usage":null}

data: {"id":"chatcmpl-a18b42fb-2084-4da5-99c0-467fdb5a1ae1","choices":[{"index":0,"delta":{"content":" intelligence","function_call":null,"tool_calls":null,"role":"assistant","refusal":null,"reasoning_content":null}}],"created":1768516782,"model":"Qwen/Qwen2.5-1.5B-Instruct",

100 36408    0 36240  100   168  56423    261 --:--:-- --:--:-- --:--:-- 56710


CompletedProcess(args=['curl', 'http://172.27.37.39:30100/v1/chat/completions', '-H', 'Content-Type: application/json', '-d', '{\n    "model": "Qwen/Qwen2.5-1.5B-Instruct",\n    "messages": [{"role": "user", "content": "Write a short poem about AI"}],\n    "stream": true,\n    "max_tokens": 100\n  }'], returncode=0)

### Step 5: Test with Different Parameters


In [31]:
import subprocess

# Get node IP
result = subprocess.run(
    ['kubectl', 'get', 'nodes', '-o', 'jsonpath={.items[0].status.addresses[?(@.type=="InternalIP")].address}'],
    capture_output=True,
    text=True
)
node_ip = result.stdout.strip()

# Test with different parameters
subprocess.run([
    'curl', f'http://{node_ip}:30100/v1/chat/completions',
    '-H', 'Content-Type: application/json',
    '-d', '''{
    "model": "Qwen/Qwen2.5-1.5B-Instruct",
    "messages": [{"role": "user", "content": "Explain quantum computing in one sentence"}],
    "stream": false,
    "temperature": 0.7,
    "max_tokens": 100,
    "top_p": 0.9
  }'''
])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

{"id":"chatcmpl-21d5d070-5835-430e-8a19-5b1e3a65bd70","choices":[{"index":0,"message":{"content":"Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data.","role":"assistant","reasoning_content":null},"finish_reason":"stop"}],"created":1768516796,"model":"Qwen/Qwen2.5-1.5B-Instruct","object":"chat.completion","usage":{"prompt_tokens":36,"completion_tokens":32,"total_tokens":68}}

100   701  100   476  100   225   2279   1077 --:--:-- --:--:-- --:--:--  3354


CompletedProcess(args=['curl', 'http://172.27.37.39:30100/v1/chat/completions', '-H', 'Content-Type: application/json', '-d', '{\n    "model": "Qwen/Qwen2.5-1.5B-Instruct",\n    "messages": [{"role": "user", "content": "Explain quantum computing in one sentence"}],\n    "stream": false,\n    "temperature": 0.7,\n    "max_tokens": 100,\n    "top_p": 0.9\n  }'], returncode=0)

## Section 5: Benchmarking with AI-Perf

### Objectives
- Install and configure AI-Perf benchmarking tool
- Run performance benchmarks against your Kubernetes deployment
- Analyze throughput, latency, and token metrics
- Compare performance across different configurations

### Metrics to Measure
- Throughput (requests/second, tokens/second)
- Latency (TTFT - Time To First Token, TPOT - Time Per Output Token, end-to-end)
- GPU utilization
- KV cache efficiency

### Benchmarking Setup
You'll run AI-Perf from your local machine against the port-forwarded service, simulating:
- Different concurrency levels (fixed concurrent requests)
- Request rate patterns (requests per second)
- Various workload characteristics

### Step 1: Install AI-Perf (if not already installed)


In [4]:
# Install AI-Perf benchmarking tool
!uv pip install aiperf -q
print("✓ AI-Perf installed")

/bin/bash: line 1: uv: command not found
✓ AI-Perf installed


### Step 2: Run Baseline Benchmark (Low Concurrency)


In [ ]:
import subprocess

# Get node IP
result = subprocess.run(
    ['kubectl', 'get', 'nodes', '-o', 'jsonpath={.items[0].status.addresses[?(@.type=="InternalIP")].address}'],
    capture_output=True,
    text=True
)
node_ip = result.stdout.strip()

print("Running low concurrency benchmark...")

# Run a simple benchmark with low concurrency
subprocess.run([
    'aiperf', 'profile',
    '--log-level', 'warning',
    '--model', 'Qwen/Qwen2.5-1.5B-Instruct',
    '--url', f'http://{node_ip}:30100',
    '--endpoint-type', 'chat',
    '--streaming',
    '--concurrency', '1',
    '--request-count', '100'
])

print("\n✓ Baseline benchmark complete")

[>1u[?2048$p                                 NVIDIA AIPerf                                  
╭────────────────────────────── Application Logs ──────────────────────────────╮
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│             



                                    NVIDIA AIPerf | LLM Metrics                                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━┓
┃                              Metric ┃    avg ┃    min ┃    max ┃    p99 ┃    p90 ┃    p50 ┃  std ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━┩
│            Time to First Token (ms) │  36.03 │  21.03 │  43.53 │  38.37 │  36.81 │  36.07 │ 1.80 │
│           Time to Second Token (ms) │   0.11 │   0.03 │   6.05 │   0.13 │   0.06 │   0.05 │ 0.60 │
│                Request Latency (ms) │ 113.08 │  87.59 │ 138.30 │ 122.68 │ 114.69 │ 113.26 │ 5.16 │
│            Inter Token Latency (ms) │   5.17 │   4.64 │   6.77 │   6.38 │   5.24 │   5.15 │ 0.23 │
│    Output Token Throughput Per User │ 193.59 │ 147.71 │ 215.51 │ 212.76 │ 196.15 │ 194.03 │ 7.46 │
│                   (tokens/sec/user) │        │        │        │        │        │     

[<u

### Step 3: Run Benchmark with Higher Concurrency


In [ ]:
import subprocess

# Get node IP
result = subprocess.run(
    ['kubectl', 'get', 'nodes', '-o', 'jsonpath={.items[0].status.addresses[?(@.type=="InternalIP")].address}'],
    capture_output=True,
    text=True
)
node_ip = result.stdout.strip()

print("Running high concurrency benchmark...")

# Test with higher concurrency to stress test
subprocess.run([
    'aiperf', 'profile',
    '--log-level', 'warning',
    '--model', 'Qwen/Qwen2.5-1.5B-Instruct',
    '--url', f'http://{node_ip}:30100',
    '--endpoint-type', 'chat',
    '--streaming',
    '--concurrency', '4',
    '--request-count', '200'
])

print("\n✓ High concurrency benchmark complete")

[>1u[?2048$p                                 NVIDIA AIPerf                                  
╭────────────────────────────── Application Logs ──────────────────────────────╮
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│             



                                    NVIDIA AIPerf | LLM Metrics                                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃                             Metric ┃    avg ┃    min ┃    max ┃    p99 ┃    p90 ┃    p50 ┃   std ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│           Time to First Token (ms) │  27.01 │  11.43 │  58.97 │  51.22 │  48.53 │  18.14 │ 15.79 │
│          Time to Second Token (ms) │   4.95 │   0.03 │  28.29 │  13.73 │   7.58 │   6.73 │  4.21 │
│               Request Latency (ms) │ 130.19 │  88.19 │ 155.33 │ 155.00 │ 149.53 │ 119.54 │ 15.24 │
│           Inter Token Latency (ms) │   6.94 │   4.74 │   8.51 │   8.36 │   7.81 │   6.93 │  0.52 │
│   Output Token Throughput Per User │ 144.89 │ 117.52 │ 210.91 │ 176.07 │ 156.38 │ 144.26 │ 11.07 │
│                  (tokens/sec/user) │        │        │        │        │        │      

### Step 4: Run Benchmark with Request Rate


In [ ]:
import subprocess

# Get node IP
result = subprocess.run(
    ['kubectl', 'get', 'nodes', '-o', 'jsonpath={.items[0].status.addresses[?(@.type=="InternalIP")].address}'],
    capture_output=True,
    text=True
)
node_ip = result.stdout.strip()

print("Running request rate benchmark...")

# Test with request rate instead of concurrency
subprocess.run([
    'aiperf', 'profile',
    '--log-level', 'warning',
    '--model', 'Qwen/Qwen2.5-1.5B-Instruct',
    '--url', f'http://{node_ip}:30100',
    '--endpoint-type', 'chat',
    '--streaming',
    '--request-rate', '10',
    '--request-count', '200'
])

print("\n✓ Request rate benchmark complete")

[>1u[?2048$p                                 NVIDIA AIPerf                                  
╭────────────────────────────── Application Logs ──────────────────────────────╮
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│                                                                              │
│             



                                    NVIDIA AIPerf | LLM Metrics                                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┓
┃                             Metric ┃    avg ┃    min ┃    max ┃    p99 ┃    p90 ┃    p50 ┃   std ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━┩
│           Time to First Token (ms) │  17.12 │  10.51 │  47.92 │  30.89 │  23.21 │  17.05 │  5.05 │
│          Time to Second Token (ms) │   6.77 │   0.07 │  13.79 │  12.81 │   7.09 │   6.55 │  1.36 │
│               Request Latency (ms) │ 118.89 │  82.69 │ 154.20 │ 149.51 │ 134.08 │ 116.45 │ 11.60 │
│           Inter Token Latency (ms) │   6.90 │   5.77 │   8.95 │   8.20 │   7.73 │   6.75 │  0.49 │
│   Output Token Throughput Per User │ 145.68 │ 111.76 │ 173.26 │ 158.16 │ 155.75 │ 148.18 │  9.63 │
│                  (tokens/sec/user) │        │        │        │        │        │      

[<u

### Step 5: Analyze Results

Review the benchmark outputs above. Key metrics to look for:
- **Throughput**: requests/second and tokens/second
- **TTFT (Time To First Token)**: How quickly does the first token appear?
- **TPOT (Time Per Output Token)**: Generation speed
- **End-to-end latency**: Total request time


### Cleanup

When you're done with Lab 1, clean up your deployment:


In [ ]:
import subprocess
import os

namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')

# Delete the deployment
subprocess.run(['kubectl', 'delete', 'dynamographdeployment', 'vllm-agg-router', '-n', namespace])

print("")
print("✓ Deployment deleted")
print("Verifying pods are terminating:")
subprocess.run(['kubectl', 'get', 'pods', '-n', namespace])

**Note:** Keep your namespace and platform for Lab 2! Only delete the deployment, not the namespace.

## Troubleshooting

### Check Pod Status


In [ ]:
import subprocess
import os

namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')

# Check all pods in your namespace
subprocess.run(['kubectl', 'get', 'pods', '-n', namespace])

print("\n# To describe a specific pod to see errors:")
print(f"# kubectl describe pod <pod-name> -n {namespace}")

### View Pod Logs


In [ ]:
import subprocess
import os

namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')

# View logs from a specific component
print("Frontend logs:")
subprocess.run(['kubectl', 'logs', '-l', 'component=Frontend', '-n', namespace, '--tail=50'])

print("\nWorker logs:")
subprocess.run(['kubectl', 'logs', '-l', 'component=VllmDecodeWorker', '-n', namespace, '--tail=50'])

### Check Deployment Status


In [ ]:
import subprocess
import os

namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')

# Check DynamoGraphDeployment status
print("DynamoGraphDeployment status:")
subprocess.run(['kubectl', 'describe', 'dynamographdeployment', 'vllm-agg-router', '-n', namespace])

print("\nOperator logs:")
subprocess.run(['kubectl', 'logs', '-l', 'app.kubernetes.io/name=dynamo-operator', '-n', namespace, '--tail=50'])

### Check Recent Events


In [ ]:
import subprocess
import os

namespace = os.environ.get('NAMESPACE', 'dynamo-lab1')

# View recent events in your namespace
result = subprocess.run(
    ['kubectl', 'get', 'events', '-n', namespace, '--sort-by=.lastTimestamp'],
    capture_output=True,
    text=True
)

# Show last 20 lines
lines = result.stdout.split('\n')
for line in lines[-20:]:
    if line:
        print(line)

### Common Issues

1. **ImagePullBackOff**: Check if you have access to NGC containers. Verify image version is correct.
2. **Pods stuck in Pending**: Check if GPU resources are available: `kubectl describe pod <pod-name> -n $NAMESPACE`
3. **Model download slow**: First run takes longer due to model download. Check worker logs for progress.
4. **Port forward not working**: Make sure pods are `1/1 Ready` before forwarding. Kill existing port-forward processes: `pkill -f port-forward`

---

## Summary

### What You Learned
- ✅ How to set up a namespace-scoped Dynamo deployment on Kubernetes
- ✅ Kubernetes-based aggregated deployment architecture
- ✅ Creating and managing DynamoGraphDeployment resources
- ✅ Backend engine deployment (vLLM)
- ✅ Testing with OpenAI-compatible API
- ✅ Performance benchmarking with AI-Perf

### Key Takeaways
- Namespace-scoped operators enable safe multi-tenant deployments
- Aggregated serving is simpler to deploy and suitable for single-node models
- KV-cache routing provides intelligent load balancing across replicas
- DynamoGraphDeployment CRD simplifies complex inference deployments
- AI-Perf provides comprehensive performance insights

### Next Steps
- **(Optional)** Complete the **Monitoring Extension** (`lab1-monitoring.md`) to set up Prometheus and Grafana for observability
- In **Lab 2**, you'll explore disaggregated serving with separate prefill and decode workers, and use AIConfigurator to optimize configurations for larger models

---

## Appendix: Step-by-Step Commands

This appendix provides complete commands for each section. Use these as a reference during the lab.

**Note for MicroK8s users:** Replace `kubectl` with `microk8s kubectl` in all commands below, or set up an alias:


In [ ]:
%%python
alias kubectl='microk8s kubectl'

### A1. Environment Setup


In [ ]:
%%python
# Verify kubectl is installed and configured
kubectl version --client
kubectl cluster-info

# Set your configuration (customize with your name!)
export NAMESPACE="dynamo-yourname"  # Replace 'yourname' with your actual name
export RELEASE_VERSION="0.5.0"     # Dynamo version
export HF_TOKEN="your_hf_token"    # Your HuggingFace token
export CACHE_PATH="/data/huggingface-cache"  # Shared cache path (ask instructor)

# Create your personal namespace
kubectl create namespace ${NAMESPACE}

# Verify namespace was created
kubectl get namespace ${NAMESPACE}

# Check GPU nodes are available (optional)
kubectl get nodes -o custom-columns=NAME:.metadata.name,GPUs:.status.capacity.nvidia\\.com/gpu

### A2. Install Dynamo Platform (Namespace-Scoped)


In [ ]:
%%python
# Step 1: Check if CRDs are already installed (cluster-wide)
if kubectl get crd dynamographdeployments.nvidia.com &>/dev/null && \
   kubectl get crd dynamocomponentdeployments.nvidia.com &>/dev/null; then
    echo "✓ CRDs already installed"
else
    echo "⚠️  CRDs not found. Ask instructor to install them, or run:"
    echo "helm fetch https://helm.ngc.nvidia.com/nvidia/ai-dynamo/charts/dynamo-crds-${RELEASE_VERSION}.tgz"
    echo "helm install dynamo-crds dynamo-crds-${RELEASE_VERSION}.tgz --namespace default"
fi

# Step 2: Download Dynamo platform helm chart
helm fetch https://helm.ngc.nvidia.com/nvidia/ai-dynamo/charts/dynamo-platform-${RELEASE_VERSION}.tgz

# Step 3: Install namespace-scoped Dynamo platform
# IMPORTANT: --set dynamo-operator.namespaceRestriction.enabled=true restricts operator to this namespace
helm install dynamo-platform dynamo-platform-${RELEASE_VERSION}.tgz \
  --namespace ${NAMESPACE} \
  --set dynamo-operator.namespaceRestriction.enabled=true

# Step 4: Wait for platform pods to be ready (~2-3 minutes)
echo "Waiting for platform pods to be ready..."
kubectl wait --for=condition=ready pod \
  --all \
  --namespace ${NAMESPACE} \
  --timeout=300s

# Step 5: Verify platform is running
kubectl get pods -n ${NAMESPACE}
# You should see: dynamo-operator, etcd, and nats pods in Running state

# Step 6: Create HuggingFace token secret
kubectl create secret generic hf-token-secret \
  --from-literal=HF_TOKEN="${HF_TOKEN}" \
  --namespace ${NAMESPACE}

# Verify secret was created
kubectl get secret hf-token-secret -n ${NAMESPACE}

### A3. Deploy Your First Model

Create a deployment YAML file `agg_router.yaml`:

```yaml
# agg_router.yaml
apiVersion: nvidia.com/v1alpha1
kind: DynamoGraphDeployment
metadata:
  name: vllm-agg-router
spec:
  services:
    Frontend:
      dynamoNamespace: vllm-agg-router
      componentType: frontend
      replicas: 1
      extraPodSpec:
        mainContainer:
          image: nvcr.io/nvidia/ai-dynamo/vllm-runtime:0.5.0
      envs:
        - name: DYN_ROUTER_MODE
          value: kv
    VllmDecodeWorker:
      envFromSecret: hf-token-secret
      dynamoNamespace: vllm-agg-router
      componentType: worker
      replicas: 1
      resources:
        limits:
          gpu: "1"
      envs:
        - name: DYN_LOG
          value: "info"
      extraPodSpec:
        volumes:
        - name: local-model-cache
          hostPath:
            path: /data/huggingface-cache  # Update if instructor provides different path
            type: DirectoryOrCreate
        mainContainer:
          image: nvcr.io/nvidia/ai-dynamo/vllm-runtime:0.5.0
          volumeMounts:
          - name: local-model-cache
            mountPath: /root/.cache
          workingDir: /workspace/components/backends/vllm
          command:
            - /bin/sh
            - -c
          args:
            - python3 -m dynamo.vllm --model Qwen/Qwen2.5-1.5B-Instruct
```

Deploy the model:


In [ ]:
%%python
# Apply the deployment
kubectl apply -f agg_router.yaml --namespace ${NAMESPACE}

# Monitor deployment progress
kubectl get dynamographdeployment -n ${NAMESPACE}

# Watch pods starting up (this takes 4-6 minutes for first run)
kubectl get pods -n ${NAMESPACE} -w
# Press Ctrl+C to stop watching

# Check specific pod status
kubectl get pods -n ${NAMESPACE} | grep vllm

# View worker logs to see model loading progress
WORKER_POD=$(kubectl get pods -n ${NAMESPACE} | grep vllmdecodeworker | head -1 | awk '{print $1}')
kubectl logs ${WORKER_POD} -n ${NAMESPACE} --tail=50 --follow

### A4. Test the Deployment


In [ ]:
import subprocess

# The frontend is exposed via NodePort on port 30100
# Get the node IP
result = subprocess.run(
    ['kubectl', 'get', 'nodes', '-o', 'jsonpath={.items[0].status.addresses[?(@.type=="InternalIP")].address}'],
    capture_output=True,
    text=True
)
node_ip = result.stdout.strip()
print(f"Frontend URL: http://{node_ip}:30100")
print("\nQuick test commands (run in terminal):\n")
print(f"# Test 1: Check available models")
print(f"curl http://{node_ip}:30100/v1/models\n")
print(f"# Test 2: Simple chat completion")
print(f"curl http://{node_ip}:30100/v1/chat/completions -H 'Content-Type: application/json' -d '{{\"model\": \"Qwen/Qwen2.5-1.5B-Instruct\", \"messages\": [{{\"role\": \"user\", \"content\": \"Hello!\"}}], \"stream\": false, \"max_tokens\": 50}}'")

### A5. Benchmark with AI-Perf


In [ ]:
import subprocess

# Get node IP
result = subprocess.run(
    ['kubectl', 'get', 'nodes', '-o', 'jsonpath={.items[0].status.addresses[?(@.type=="InternalIP")].address}'],
    capture_output=True,
    text=True
)
node_ip = result.stdout.strip()
frontend_url = f"http://{node_ip}:30100"

print(f"Benchmarking frontend at: {frontend_url}\n")

# Install AI-Perf (if not already installed)
subprocess.run(['pip', 'install', 'aiperf'], capture_output=True)

print("=== Running benchmarks ===\n")

# Run a simple benchmark (adjust parameters as needed)
print("1. Low concurrency benchmark...")
subprocess.run([
    'aiperf', 'profile',
    '--log-level', 'warning',
    '--model', 'Qwen/Qwen2.5-1.5B-Instruct',
    '--url', frontend_url,
    '--endpoint-type', 'chat',
    '--streaming',
    '--concurrency', '1',
    '--request-count', '100'
])

# Run with higher concurrency
print("\n2. High concurrency benchmark...")
subprocess.run([
    'aiperf', 'profile',
    '--log-level', 'warning',
    '--model', 'Qwen/Qwen2.5-1.5B-Instruct',
    '--url', frontend_url,
    '--endpoint-type', 'chat',
    '--streaming',
    '--concurrency', '4',
    '--request-count', '200'
])

# Run with request rate
print("\n3. Request rate benchmark...")
subprocess.run([
    'aiperf', 'profile',
    '--log-level', 'warning',
    '--model', 'Qwen/Qwen2.5-1.5B-Instruct',
    '--url', frontend_url,
    '--endpoint-type', 'chat',
    '--streaming',
    '--request-rate', '10',
    '--request-count', '200'
])

### A6. Scale Your Deployment


In [ ]:
%%python
# Edit your agg_router.yaml and change replicas from 1 to 2
# Then reapply:
kubectl apply -f agg_router.yaml --namespace ${NAMESPACE}

# Watch the new worker come online
kubectl get pods -n ${NAMESPACE} -w

# Test that load is distributed (KV-cache routing should work)
# Run multiple requests and check logs from both workers
kubectl logs -l component=VllmDecodeWorker -n ${NAMESPACE} --tail=20

### A7. Cleanup


In [ ]:
%%python
# Delete the deployment
kubectl delete dynamographdeployment vllm-agg-router -n ${NAMESPACE}

# Verify pods are terminating
kubectl get pods -n ${NAMESPACE}

# (Optional) Keep your namespace for Lab 2
# To completely clean up (only if you're done with all labs):
# kubectl delete namespace ${NAMESPACE}

### A8. Troubleshooting


In [ ]:
%%python
# Check pod status
kubectl get pods -n ${NAMESPACE}

# Describe a pod to see errors
kubectl describe pod <pod-name> -n ${NAMESPACE}

# View logs from a specific pod
kubectl logs <pod-name> -n ${NAMESPACE}

# Check DynamoGraphDeployment status
kubectl describe dynamographdeployment vllm-agg-router -n ${NAMESPACE}

# Check operator logs
kubectl logs -l app.kubernetes.io/name=dynamo-operator -n ${NAMESPACE}

# Check if image pull is working
kubectl get events -n ${NAMESPACE} --sort-by='.lastTimestamp'